In [1]:
import logging
import time
import random
from datetime import datetime

n_campi = 'CABL'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('consolidação')
logger.setLevel(logging.DEBUG)

fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_consolidacao.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {n_campi}')
# logger.debug('debug message')
# logger.warning('warn message')

In [2]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {n_campi}')
levantamento = pd.read_excel('consolidacao/levantamento/2023_levantamento_CABL.xlsx')

logger.info(f'carregando palnilha de estoque suap campus {n_campi}')
suap = pd.read_excel('consolidacao/suap/2023-10-26_09-23_CABL_relatorio_almox.xlsx')

logger.info(f'tratando dados da planilha de levantameto fisico {n_campi}')

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]


# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

# print(levantamento.info())
# print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})

# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

logger.info(f'removendo dados duplicados')
# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

logger.info(f'cruzando dados entre levantamento fisico e suap')

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
# display(suap_cross)
# display(levantamento_cross)

logger.info(f'consolidando dados')

logger.info(f'exportando planilha de consolidação')
consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

logger.info(f'consolidação campus {n_campi} finalizada')

In [3]:
logger.info(f'montando planilha de ajustes ')

consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

logger.info(f'exportando planilha de ajustes ')

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,und,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,32.03.161,"ABSORVENTE HIGIENICO , TIPO NORMAL COM ABAS , ...",UN,625,03 - MATERIAL DESTINADO A ASSISTENCIA SOCIAL,NaN,NaN,NaN,left_only,NaN
1,30.07.1103,"CAFÉ, apresentação pó, tipo embalagem a vácuo",KG,64,07 - GENEROS DE ALIMENTACAO,"CAFÉ, apresentação pó, tipo embalagem a vácuo",49.0,maria.rozimar@abreuelima.ifpe.edu.br,both,15.0
2,30.07.1462,Açucar cristal 1KG,UN,50,07 - GENEROS DE ALIMENTACAO,NaN,NaN,NaN,left_only,NaN
3,30.07.705,"ADOÇANTE ASPECTO FISICO LÍQUIDO TRANSPARENTE, ...",UN,11,07 - GENEROS DE ALIMENTACAO,"ADOÇANTE ASPECTO FISICO LÍQUIDO TRANSPARENTE, ...",11.0,maria.rozimar@abreuelima.ifpe.edu.br,both,0.0
4,30.07.835,"AÇÚCAR, TIPO CRISTAL, PACOTE DE 1KG.",UN,1,07 - GENEROS DE ALIMENTACAO,"AÇÚCAR, TIPO CRISTAL, PACOTE DE 1KG.",50.0,maria.rozimar@abreuelima.ifpe.edu.br,both,-49.0
...,...,...,...,...,...,...,...,...,...,...
139,30.36.466,"SACO PLÁSTICO LIXO, CAPACIDADE 30 L, COR BRANC...",UN,1200,36 - MATERIAL HOSPITALAR,Saco para Lixo Infectante Branco 30 Litros com...,22.0,thiago.galdino@abreuelima.ifpe.edu.br,both,1178.0
140,30.36.467,"DISPENSER HIGIENIZADOR, MATERIAL ACRÍLICO, CAP...",UN,1,36 - MATERIAL HOSPITALAR,"DISPENSER HIGIENIZADOR, MATERIAL plástico, CAP...",1.0,thiago.galdino@abreuelima.ifpe.edu.br,both,0.0
141,30.36.470,"BOMBONA, MATERIAL PLÁSTICO, CAPACIDADE 5 L, AP...",UN,20,36 - MATERIAL HOSPITALAR,"BOMBONA, MATERIAL PLÁSTICO, CAPACIDADE 5 L, AP...",20.0,thiago.galdino@abreuelima.ifpe.edu.br,both,0.0
142,30.36.473,"DISPENSER PAPEL TOALHA, MATERIAL AÇO, TIPO INT...",UN,9,36 - MATERIAL HOSPITALAR,"DISPENSER PAPEL TOALHA, MATERIAL plástico, TIP...",9.0,thiago.galdino@abreuelima.ifpe.edu.br,both,0.0


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
1,30.07.1103,07 - GENEROS DE ALIMENTACAO,"CAFÉ, apresentação pó, tipo embalagem a vácuo",64,49.0,15.0,pedido no suap
3,30.07.705,07 - GENEROS DE ALIMENTACAO,"ADOÇANTE ASPECTO FISICO LÍQUIDO TRANSPARENTE, ...",11,11.0,0.0,adequado
4,30.07.835,07 - GENEROS DE ALIMENTACAO,"AÇÚCAR, TIPO CRISTAL, PACOTE DE 1KG.",1,50.0,-49.0,saida sem pedido
5,30.16.1026,16 - MATERIAL DE EXPEDIENTE,"GRAMPEADOR DE METAL, TIPO PROFISSIONAL CAPACID...",1,2.0,-1.0,saida sem pedido
6,30.16.1060,16 - MATERIAL DE EXPEDIENTE,PINCEL ATOMICO COR AZUL,9,9.0,0.0,adequado
...,...,...,...,...,...,...,...
139,30.36.466,36 - MATERIAL HOSPITALAR,"SACO PLÁSTICO LIXO, CAPACIDADE 30 L, COR BRANC...",1200,22.0,1178.0,pedido no suap
140,30.36.467,36 - MATERIAL HOSPITALAR,"DISPENSER HIGIENIZADOR, MATERIAL ACRÍLICO, CAP...",1,1.0,0.0,adequado
141,30.36.470,36 - MATERIAL HOSPITALAR,"BOMBONA, MATERIAL PLÁSTICO, CAPACIDADE 5 L, AP...",20,20.0,0.0,adequado
142,30.36.473,36 - MATERIAL HOSPITALAR,"DISPENSER PAPEL TOALHA, MATERIAL AÇO, TIPO INT...",9,9.0,0.0,adequado
